In [1]:
%pip install tensorflow gensim numpy

  Using cached tensorflow-2.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached gensim-4.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.2 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-3.1.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import gensim.downloader as api
from tensorflow.keras.layers import Embedding
import numpy as np

tf.random.set_seed(123)

In [ ]:
%pip install pandas

In [5]:
import pandas as pd

In [2]:
# Downloasd dataset

!gdown https://drive.google.com/file/d/1DNgciEZTQWuo_wq4qMGIV8z3VC1Onjjv/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1ePfUhb2XeicnybAMu0y2w3BhiWm4XY8t/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1FRn6Gn3ijmCWfhUI6QkGm4g4LeQ7Veg0/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1Fw3yivW_PSH-C5T_4-9bWshxPHxvxtAi/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1F9r26d7emj43EG6kcMfvOsu1rI8yxR4d/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1I0vjwcnwfaMZdwhS-oJjqUoihl9NxZ1g/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1tX9mZBStZGCQmOhgaCu-tSdlXqQU77ou/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1-GNcBpiB3Y0RzbJLawxL31IjkTBtpUz2/view?usp=share_link --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1DNgciEZTQWuo_wq4qMGIV8z3VC1Onjjv
To: /home/ashok/aspect_based_sentiment_analysis/laptop_dev.csv
100%|███████████████████████████████████████| 46.5k/46.5k [00:00<00:00, 812kB/s]
Downloading...
From: https://drive.google.com/uc?id=1ePfUhb2XeicnybAMu0y2w3BhiWm4XY8t
To: /home/ashok/aspect_based_sentiment_analysis/laptop_test.csv
100%|██████████████████████████████████████| 98.6k/98.6k [00:00<00:00, 1.08MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FRn6Gn3ijmCWfhUI6QkGm4g4LeQ7Veg0
To: /home/ashok/aspect_based_sentiment_analysis/laptop_train.csv
100%|████████████████████████████████████████| 378k/378k [00:00<00:00, 2.45MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fw3yivW_PSH-C5T_4-9bWshxPHxvxtAi
To: /home/ashok/aspect_based_sentiment_analysis/mams_test.csv
100%|████████████████████████████████████████| 222k/222k [00:00<00:00, 1.73MB/s]
Downloading...
From: https://drive.google.com/uc?id=1F9r26d7emj43EG6kcMfvOsu1r

In [6]:
def load_mams_data(path):
  df = pd.read_csv(path)
  df = df[['text', 'term', 'polarity']]
  return df

def load_laptop_data(path):
  text_ds = []
  term_ds = []
  polarity_ds = []
  with open(path, 'r') as f:
    data = f.read().splitlines()

  terms = []
  polarities = []
  text = ""
  term = ""
  polarity = ""
  for row in data[2:]:
    # We are on new text column
    if len(row) == 1:
      if term != "":
        terms.append(term)
        polarities.append(polarity)
      for i in range(len(terms)):
        text_ds.append(text.strip())
        term_ds.append(terms[i].strip())
        polarity_ds.append(polarities[i])

      text = ""
      term = ""
      polarity = ""
      terms = []
      polarities = []

    else:
      token, label = row.rsplit(',', 1)
      text += " " + token

      if label == 'T-POS':
        label = 'positive'
      elif label == 'T-NEG':
        label = 'negative'
      elif label == 'T-NEU':
        label = 'neutral'
      else:
        if term != "":
          terms.append(term)
          polarities.append(polarity)
        term = ""
        polarity = ""
        continue

      if polarity == label:
        term += " " + token

      else:
        if term != "":
          terms.append(term)
          polarities.append(polarity)

        polarity = label
        term = token

  df = pd.DataFrame()
  df['text'] = text_ds
  df['term'] = term_ds
  df['polarity'] = polarity_ds
  return df

In [7]:
laptop_train, laptop_test, laptop_valid = list(map(load_laptop_data, ['laptop_train.csv', 'laptop_test.csv', 'laptop_dev.csv']))

# restaurant dataset
rest_train, rest_test, rest_valid = list(map(load_laptop_data, ['rest16_train.csv', 'rest16_test.csv', 'rest16_dev.csv']))

# mams dataset
mams_train, mams_test, mams_valid = list(map(load_mams_data, ['mams_train.csv', 'mams_test.csv', 'mams_val.csv']))

In [8]:
# dataset combined, shuffled
train_data = pd.concat([laptop_train, rest_train, mams_train, laptop_test, rest_test, mams_test]).sample(frac = 1).reset_index(drop = True)
valid_data = pd.concat([laptop_valid, rest_valid, mams_valid]).sample(frac = 1).reset_index(drop = True)

In [9]:
print('*'*30,'data ', '*'*30)
train_data.head()

****************************** data  ******************************


,text,term,polarity
0,"last time, the waiter told my roommate he'd ha...",waiter,negative
1,After ordering my dinner and having it come to...,mushrooms,negative
2,"sort of crouched in a chair, leaning over a ta...",table,neutral
3,"It gets stuck all of the time you use it "","" a...",work,negative
4,The kitchen would do well to offer more desser...,Rice connotes,positive


In [10]:
train_data = train_data[train_data['polarity'] != 'neutral']

valid_data = valid_data[valid_data['polarity'] != 'neutral']

In [11]:
# data processing, encoding labels, and lower casing the sentence text and aspect text

pol2idx = { 'positive' : 1, 'negative' : 0}
for data in [train_data, valid_data]:
  data['polarity'] = data['polarity'].apply(lambda x:pol2idx[x])
  data['text'] = data['text'].apply(lambda x: x.lower())
  data['term'] = data['term'].apply(lambda x: str(x).lower()).astype('str')

  data = data.drop_duplicates(["text", "term", "polarity"])

train_data.head()

,text,term,polarity
0,"last time, the waiter told my roommate he'd ha...",waiter,0
1,after ordering my dinner and having it come to...,mushrooms,0
3,"it gets stuck all of the time you use it "","" a...",work,0
4,the kitchen would do well to offer more desser...,rice connotes,1
6,park your trench coat at the coat check and be...,cuisine,1


In [12]:
import gensim.downloader as api
# load word embeddings from gensim api

word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [13]:
print(api.BASE_DIR)

/home/ashok/gensim-data
